In [286]:
from importlib import reload
import util
reload(util)
data = util.FontData
FontDataset = util.FontDataset
import experiment
reload(experiment)
from sklearn import linear_model, neural_network

In [287]:
data.load()

Loading embeddings...done
Loading geometric + semantic vectors...done
Loading images...done


In [288]:
train_dataset = FontDataset(data, 'train')
val_dataset = FontDataset(data, 'val')

In [289]:
train_geometric = data.get_all_geometric('train')
val_geometric = data.get_all_geometric('val')

In [290]:
train_image = data.get_all_image('train')
train_image = train_image.reshape(train_image.shape[0], -1)
val_image = data.get_all_image('val')
val_image = val_image.reshape(val_image.shape[0], -1)

In [291]:
train_semantic = data.get_all_semantic('train')
val_semantic = data.get_all_semantic('val')

### Linear Regression: Image -> Geometric

In [292]:
image_geometric_reg = linear_model.LinearRegression()

In [293]:
image_geometric_reg.fit(train_image, train_geometric)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [294]:
train_stats, train_geometric_pred = experiment.run_experiment(image_geometric_reg, train_image, train_geometric)

{'RMSE': 0.1633429229259491, 'R2': 0.9738025577307982}


In [295]:
val_stats, val_geometric_pred = experiment.run_experiment(image_geometric_reg, val_image, val_geometric)

{'RMSE': 34.59562683105469, 'R2': -1202.1584194240093}


### Linear Regression: Geometric -> Semantic

In [296]:
geometric_semantic_reg = linear_model.LinearRegression()

In [297]:
geometric_semantic_reg.fit(train_geometric, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [298]:
train_stats, train_semantic_pred = experiment.run_experiment(geometric_semantic_reg, train_geometric, train_semantic)

{'RMSE': 0.7227101922035217, 'R2': 0.46196948404285115}


In [299]:
val_stats, val_semantic_pred = experiment.run_experiment(geometric_semantic_reg, val_geometric, val_semantic)

{'RMSE': 0.7563706040382385, 'R2': 0.4712526857284771}


### Linear Regressions: Image -> Geometric -> Semantic

In [300]:
class SequentialLinearRegression:
    def __init__(self, reg1, reg2):
        self.reg1 = reg1
        self.reg2 = reg2
    def predict(self, X):
        return self.reg2.predict(self.reg1.predict(X))

image_geometric_semantic_reg = SequentialLinearRegression(image_geometric_reg, geometric_semantic_reg)

In [301]:
train_stats, train_semantic_pred = experiment.run_experiment(image_geometric_semantic_reg, train_image, train_semantic)

{'RMSE': 0.73167884349823, 'R2': 0.4485337055649404}


In [302]:
val_stats, val_semantic_pred = experiment.run_experiment(image_geometric_semantic_reg, val_image, val_semantic)

{'RMSE': 24.477190017700195, 'R2': -564.7180186866788}


### Linear Regression: Image -> Semantic

In [303]:
image_semantic_reg = linear_model.LinearRegression()

In [304]:
image_semantic_reg.fit(train_image, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [305]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_reg, train_image, train_semantic)

{'RMSE': 0.16490739583969116, 'R2': 0.9719962841065443}


In [306]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_reg, val_image, val_semantic)

{'RMSE': 51.0186767578125, 'R2': -2412.3122592328114}


### Multilayer Perceptron: Image -> Semantic

In [307]:
image_semantic_nn = neural_network.MLPRegressor(hidden_layer_sizes=(10,))

In [308]:
image_semantic_nn.fit(train_image, train_semantic)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(10,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [309]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_nn, train_image, train_semantic)

{'RMSE': 0.987125039100647, 'R2': -0.003841976517193592}


In [310]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_nn, val_image, val_semantic)

{'RMSE': 1.040409803390503, 'R2': -0.007754028798483221}


In [311]:
val_semantic_pred[0]

tensor([ 0.0104, -0.0278, -0.0854,  0.0016, -0.1280,  0.0253,  0.0964, -0.0101,
         0.1541, -0.0994, -0.0898, -0.0272, -0.0257,  0.0183, -0.0265,  0.0309,
         0.0183, -0.0824, -0.0036, -0.0251, -0.0086, -0.0291,  0.1042, -0.0182,
        -0.0384, -0.1008,  0.0392, -0.0236, -0.0048, -0.0424, -0.0021])

In [312]:
data.get_semantic(data.get_name(0, "val"))

array([ 1.38946555,  2.07786678,  0.53825864,  1.0735803 ,  0.94702023,
       -0.73411685, -0.29449533,  1.74432709,  1.74957114,  2.14193563,
        2.34482157,  2.12647931,  1.96046724, -1.6338738 , -0.44899942,
        1.59132233, -0.04400323,  1.32978186,  2.119893  , -1.84771722,
        2.08848176,  2.42672959,  1.56701499, -0.50614695,  1.88594378,
        0.82936429, -1.63937803,  1.45262655,  1.31072242, -0.3115636 ,
       -2.23565785])